# Riverland

### PROBLEM 

There is a country with a large river system. The cities in this country are numbered 1 through N; city number 1 is the capital. The river system forms a rooted tree such that the capital is the root. Each river connects two cities and flows towards the root. For each non-root city i, there is a river that flows from this city to city pi. There is also a port in each city.

In the country, there are only two ways to travel between cities: by boat and by ferry. A boat can only move downstream (that is, towards the capital), while a ferry can move in both directions. There are direct ferry routes from every port to every other port.

Initially, all the ports are open. Sometimes, some ports can close or open again (possibly multiple times); the port in the capital is always open. It is not allowed to travel by ferry from a closed port, but travelling to a closed port is allowed. Travelling by boat is always possible, even from a closed port.

Sometimes, Chef wants to go on a trip. He has already chosen a city v where he will start the trip. However, he only has enough money for at most one trip by ferry. Therefore, he wants to make a trip as follows:

Choose a city b with an opened port and travel to this city by boat. It must be possible to travel from v to b by boat (specifically, v=b is allowed).
Decide to either continue from city b by ferry or end the trip in this city.
If he decides to continue by ferry, choose a city f and travel from b to f by ferry. It must also be possible to travel from b to f by ferry.
It is not allowed to travel along any river twice (regardless of direction), i.e. when Chef decides to continue by ferry, the shortest paths between (v, b) and (b, f) must have no common rivers.
Chef likes travelling by ferry, so the more rivers he visits (travels along) on a ferry, the happier he becomes. Also, each city has a fixed beauty; let's denote the beauty of city i by ai. Then, for a trip with value T, let's define the happiness Chef gains as ab+D⋅T, where D is the length of the ferry route (the number of rivers visited when travelling by ferry — possibly zero).

You should process Q queries. There are three types of queries:

- v: The port in city v closes. It is guaranteed that the port in city v was open before this query.
+ v: The port in city v reopens. It is guaranteed that the port in city v was closed before this query.
? v T: Chef wants to make a trip with value T from city v.
For each query of the third type, find the maximum happiness Chef can gain from such a trip.

### INPUT 

* The first line of the input contains two space-separated integers N and Q.
* The second line contains N−1 space-separated integers p2,p3,…,pN.
* The third line contains N space-separated integers a1,a2,…,aN.
* The following Q lines describe queries. Each of these lines starts with a character c denoting the type of the current query, a space and an integer v. If c is '?', this is followed by a space and an integer T.

### OUTPUT 

For each query of the third type, print a single line containing one integer — the maximum obtainable happiness.

### CONSTRAINT 

### EXAMPLE INPUT 

### EXAMPLE OUTPUT 

### EXPLAINATION 

In the first query, Chef travels by ferry from city 4 to city 7.

In the second query, Chef cannot travel by ferry from city 4, so he has to use the second best route, which is from city 2 to city 7.

In the third query, Chef travels to city 8 by boat and then to city 7 by ferry.

In the fourth query, Chef travels to the capital (city number 1) by boat and does not travel by ferry at all.

## SOLUTION 

In [1]:
import pandas as pd
import numpy as np

In [2]:
class Mapping(object):

    def __init__(self, nodes, p, happiness):
        self.nodes = nodes
        self.p = p
        self.happiness = happiness
        self.adj_mat = None
        self.adj_mat_ferry = None
        self.b = None
        self.f = None
        self.sel_rout = None
        self.source = None
        self.boat_matrix = None
        self.ferry_matrix = None



    def happiness_table(self):
        dic = {
            'city': self.nodes,
            'happiness': self.happiness,
        }
        return pd.DataFrame(dic, columns=['city', 'happiness'], index=self.nodes)

    def get_happiness(self, city):
        table = self.happiness_table()
        return table.loc[city]['happiness']


    def adjacency_matrix(self):
        zero_matrix = np.zeros((len(self.nodes), len(self.nodes)), np.int8)
        for i in range(len(self.nodes[1:])):
            zero_matrix[int(self.nodes[1:][i])-1, int(self.p[i])-1] = 1

        self.adj_mat = pd.DataFrame(zero_matrix, index=self.nodes, columns=self.nodes)
        return None

    def adjacency_matrix_ferry(self):
        if self.adj_mat is None:
            self.adjacency_matrix()

        self.adj_mat_ferry = self.adj_mat + self.adj_mat.transpose()
        return None

    def next_cities(self, source, ferry=True):
        if ferry is False:
            rec_mat = self.boat_matrix

        if ferry is True:
            rec_mat = self.ferry_matrix

        adj_l = rec_mat.loc[source][rec_mat.loc[source] == 1]
        if len(adj_l) > 0:
            return adj_l.index
        else:
            return []

    def routs_bfs(self, source, destination, ferry):
        """
        Routs by Brute Force (breadth first) algorithm.
        """

        journey_init = [source]
        paths_q = [journey_init]

        while len(paths_q) != 0:

            check_path = paths_q.pop()
            last_city = check_path[-1]

            if last_city == destination:
                return check_path

            for rout in self.next_cities(last_city, ferry):
                if rout not in check_path:
                    new_path = check_path + [rout]
                    paths_q.append(new_path)

        return None

    def routs_dfs(self, source, destination, paths, longest, ferry):
        """
        Routs by Brute Force (depth first) algorithm.
        """

        paths = paths+[source]

        if source == destination:
            return paths

        available_rout = self.next_cities(source, ferry)
        for rout in available_rout:
            if rout not in paths:
                if longest == None or len(paths) > len(longest):
                    new_path = self.routs_dfs(rout, destination, paths, longest, ferry)

                    if new_path != None:
                        longest = new_path

        return longest

    def longest_rout(self, start, paths, ferry, closed):
        init = [start]
        path_q = [init]
        longest = []
        while len(path_q) != 0:

            temp = path_q.pop()
            last = temp[-1]

            for i in self.next_cities(last, ferry):
                if ferry:
                    if i not in temp and i not in closed:
                        new = temp + [i]
                        path_q.append(new)
                        if len(new)>len(longest):
                            longest = new
                else:
                    if i not in temp:
                        new = temp + [i]
                        path_q.append(new)
                        if len(new)>len(longest):
                            longest = new
                            
        return longest

    def select_city(self, source, closed):

        closed = closed + [source]
        cities = self.next_cities(source, False)
        b,f = None, None
        long = []
        for i in cities:
            if self.routs_bfs(source, i, False) is not None:
                closed.append(i)
                b = i
                path = self.longest_rout(i, [], True, closed)
                if len(path) > len(long):
                    long = path
                    f = path[-1]

            else:
                return self.select_city(i, closed)

        self.b = b
        self.f = f
        self.sel_rout = long
        return None

    def set_destination(self, source):
        self.source = source
        ferry_rout = self.longest_rout(source, [], True, [])
        self.sel_rout = ferry_rout
        if len(ferry_rout) > 0:
            self.f = ferry_rout[-1]
        else:
            self.f = source

        return None

    def gain_happiness(self, source, closed, value):
        """
        ab+D⋅T,
        """
        self.source = source
        if self.adj_mat is None or self.adj_mat_ferry is None:
            self.adjacency_matrix()
            self.adjacency_matrix_ferry()

        self.boat_matrix = self.adj_mat.copy()
        self.ferry_matrix = self.adj_mat_ferry.copy()
        if len(closed)>0:
            for i in closed:
                self.ferry_matrix.loc[i]=0

        if self.source == None or self.f == None:
            self.set_destination(source)

        if self.source == self.f:
            return int(self.get_happiness(self.source))

        max_boat_rout = self.longest_rout(self.source, [], False, [])
        max_happiness = 0
        for i in max_boat_rout:
            possible_ferry_rout = self.routs_dfs(i, self.f, [], None, True)
            self.ferry_matrix[i]=0
            self.ferry_matrix.loc[i]=0
            if possible_ferry_rout != None:
                happiness = int(self.get_happiness(i)) + ((len(possible_ferry_rout)-1)*value)
            else:
                happiness = int(self.get_happiness(i))
            if happiness > max_happiness:
                max_happiness = happiness

        return max_happiness

In [6]:
def chef_happiness():
    x = input().split()
    n = list(np.arange(1, int(x[0])+1))
    p= input().split()
    a = input().split()
    mp= Mapping(n,p,a)
    close = []
    q = int(x[1])
    while q>0:
        y_d = input()
        y = y_d.split()
        if y[0] == '?':
            this_happiness = mp.gain_happiness(int(y[1]), close, int(y[2]))
            print("Output: ",this_happiness)
        elif y[0] == "-":
            close.append(int(y[1]))
        elif y[0] == "+":
            close.remove(int(y[1]))
        q -= 1

Running algorithm on sample.

In [7]:
chef_happiness()

10 9
1 2 3 2 2 6 3 8 8
30 20 6 13 8 40 7 9 13 1
? 4 11
Output:  57
- 4
? 4 11
Output:  42
- 7
? 10 6
Output:  33
+ 7
- 6
- 2
? 7 4
Output:  7
